# Swift & Value Semantics

This notebook walks through a few small examples to get a feel for consuming and producing value-semantic APIs (in the small).

## Unary functions

We begin with two functions that are approximately equal to increment an integer by `1`.

In [1]:
func foo1(_ x: Int) -> Int {
    return x + 1
}

In [2]:
func foo2(_ x: inout Int) {
    x += 1
}

Note: `inout` should be thought of as _semantically_ "copy-in-copy-out", but is implemented more efficiently.

In [3]:
var x = 1
// We now demonstrate their usage
print(x)
x = foo1(x)
print(x)
foo2(&x)  // The & helps you see where mutations are happening!
print(x)

1
2
3


In [4]:
// Of course, you can make an immutable variable, which will (as expected) not be allowed to mutate.
let y = 0

In [5]:
y = foo1(y)  // Compile error

: 

In [6]:
foo2(&y)  // Compile error

: 

## Higher-airity functions

They work exactly as you'd expect.

In [7]:
func incrementTwoInts(_ a: inout Int, _ b: inout Int) {
    a += 1
    b += 1
}

var a = 0
var b = 1
incrementTwoInts(&a, &b)
print(a)
print(b)

1
2


## Mutating Functions

Object-oriented programming taught us that we often want to group data and operations together. For example, we often like to write:

```python
myType.method()
```

instead of

```python
top_level_function(myType)
```

The extension for `inout` to "methods" is spelled `mutating`.

In [8]:
struct Foo {
    var x: Int = 0
}

In [9]:
extension Foo {
    mutating func incrementByOne() {
        x += 1
    }
}

In [10]:
func testFunction() {
    var f = Foo()
    print(f)
    f.incrementByOne()
    print(f)
}
testFunction()

Foo(x: 0)
Foo(x: 1)


Naturally, mutating functions can't be called on immutable values.

In [11]:
func testFunc2() {
    let f = Foo()
    print(f)
    f.incrementByOne()  // Compile error
}
testFunc2()

: 

Mutating functions are convenient in a variety of circumstances. For example, when using an iterator.

In [12]:
// Silly implementation to skip 2 values
func skip2<Itr: IteratorProtocol>(_ itr: inout Itr) {
    _ = itr.next()
    _ = itr.next()
}

extension IteratorProtocol {
    // Silly implementation to skip 3 values.
    mutating func skip3() {
        skip2(&self)
        _ = next()
    }
}

func doSomeIteration() {
    let r = 0...9  // Int's from 0 to 9, inclusive.
    var itr = r.makeIterator()
    print(itr.next())
    print(itr.next())
    skip2(&itr)
    print(itr.next())
    itr.skip3()
    print(itr.next())
    print(itr.next())
    print(itr.next())  // End of sequence
}
doSomeIteration()

Optional(0)
Optional(1)
Optional(4)
Optional(8)
Optional(9)
nil


## Compositions

Value semantics composes exactly as you'd expect, and continue to maintain the efficiency of in-place operations without allowing spooky-action-at-a-distance.

In [13]:
// Declare another struct composed of two additional value-semantic types.
struct Bar {
    var f: Foo
    var y: Int
}

In [14]:
extension Bar {
    init() {
        self.init(f: Foo(), y: 0)
    }
}

In [15]:
extension Bar {
    mutating func baz() {
        f.incrementByOne()
        y += 1
    }
}

In [16]:
extension Bar: CustomStringConvertible {
    public var description: String {
        "Bar(f: \(f), y: \(y))"
    }
}

In [17]:
func testFunc3() {
    var b = Bar()
    print("b: \(b)")
    b.baz()
    print("b: \(b) (post-baz())")
    let c = b
    print("c: \(c) (create copy)")
    print("b: \(b) (should be equivalent to above)")
    b.f.incrementByOne()
    print("b: \(b) (f should be incremented)")
    print("c: \(c) (should be the same as original c value)")
    b.baz()
    print("b: \(b) (more changes)")
    print("c: \(c) (more sames)")
//     c.baz() // Compile error
//     c.f.incrementByOne()  // Also compile error
}
testFunc3()

b: Bar(f: Foo(x: 0), y: 0)
b: Bar(f: Foo(x: 1), y: 1) (post-baz())
c: Bar(f: Foo(x: 1), y: 1) (create copy)
b: Bar(f: Foo(x: 1), y: 1) (should be equivalent to above)
b: Bar(f: Foo(x: 2), y: 1) (f should be incremented)
c: Bar(f: Foo(x: 1), y: 1) (should be the same as original c value)
b: Bar(f: Foo(x: 3), y: 2) (more changes)
c: Bar(f: Foo(x: 1), y: 1) (more sames)


# Swift's non-obvious properties

I've collected a few nice features of Swift that aren't apparent by reading [the language guide](https://docs.swift.org/swift-book/LanguageGuide/TheBasics.html).

Said another way, these properties of Swift enable high performance execution without sacrificing value semantics.


## Early dropping of references

Unlike C++ Swift does not mandate that local variables live until the end of their scope, so the compiler is free to destroy variables when they are no longer used.  That helps preserve the uniqueness property that allows efficient in-place mutation. Also, swift doesn't specify the memory layout of a local stack frame, which allows for a variety of optimizations, such as hoisting values to registers or tearing them apart. A few examples:

In [18]:
/// An Int, but implemented using the same lazy copy-on-write techniques as
/// arrays (and Tensors) for demonstration purposes (makes copies explicit).
public struct CoWInt {
    /// We use a class to leverage reference semantics to implement our
    /// copy-on-write type.
    ///
    /// We also keep track of the copy count for illustrative purposes.
    final class HeapStorage {
        var value: Int
        let copyCount: Int

        /// Creates a HeapStorage instance.
        init(value: Int, copyCount: Int = 0) {
            self.value = value
            self.copyCount = copyCount
        }

        /// Returns a copy of `self`, with the copyCount incremented by 1.
        func copy() -> HeapStorage {
            HeapStorage(value: value, copyCount: copyCount + 1)
        }
    }

    /// Create a CoWInt.
    public init(_ value: Int) { self.heapStorage = HeapStorage(value: value) }
    
    /// Accesses the value stored in a CoWInt.
    var value: Int {
        get { heapStorage.value }

        // Note: if we were implementing this "for real", we'd want to use
        // Swift's support for coroutines, and implement `_modify` instead.
        set {
            ensureUnique()
            heapStorage.value = newValue
        }
    }

    /// Returns the copy count of the backing storage.
    var copyCount: Int { heapStorage.copyCount }

    /// Internals below that are not marked private for demonstration purposes.
    
    /// Helper 
    /* private */
    mutating func ensureUnique() {
        if !isKnownUniquelyReferenced(&heapStorage) {
            heapStorage = heapStorage.copy()
        }
    }

    /* private */
    init(storage: HeapStorage) {
        self.heapStorage = storage
    }

    /// A handle to our heap storage.
    /* private */
    var heapStorage: HeapStorage
}

public func += (lhs: inout CoWInt, rhs: Int) {
    lhs.value += rhs
}


In [19]:
extension CoWInt: CustomStringConvertible {
    public var description: String { "CoWInt(\(value), copyCount: \(copyCount))" }
}


In [20]:
func myFunc() {
    var a = CoWInt(3)
    print(a)
    a += 6
    print(a)
}
myFunc()

CoWInt(3, copyCount: 0)
CoWInt(9, copyCount: 0)


In [21]:
func myFunc2() {
    var a = CoWInt(3)
    print(a)
    let b = a  // Make a logical copy.
    print(a)
    print(b)
    a += 3  // Actual copy happens here.
    print(a)
    print(b)
}
myFunc2()

CoWInt(3, copyCount: 0)
CoWInt(3, copyCount: 0)
CoWInt(3, copyCount: 0)
CoWInt(6, copyCount: 1)
CoWInt(3, copyCount: 0)


In [22]:
func myFunc3() {
    var a = CoWInt(3)
    var b = a
    print(a)
    a += 10
    print(a)  // Note: copy count stays zero!
}
myFunc3()

CoWInt(3, copyCount: 0)
CoWInt(13, copyCount: 0)


In [23]:
// Can always assign to an immutable binding if that's preferred.
func myFunc4() {
    var a = CoWInt(3)
    print(a)
    a += 10
    print(a)
    let b = a  // Assign to immutable binding... no actual data copy is made.
    print(b)
}
myFunc4()

CoWInt(3, copyCount: 0)
CoWInt(13, copyCount: 0)
CoWInt(13, copyCount: 0)


In [24]:
// If mutation is desired, you can always reassign to a new mutable variable.
func myFunc5() {
    var a = CoWInt(3)
    var b = a
    print(a)
    print(b)
    b += 10
    print(b)
    print(a) // Not changed!
}
myFunc5()

CoWInt(3, copyCount: 0)
CoWInt(3, copyCount: 0)
CoWInt(13, copyCount: 1)
CoWInt(3, copyCount: 0)


In [25]:
// `inout` and `mutation` help avoid copies.

extension CoWInt {
    // Functional-style update where `self` is not modified, and a new value is returned.
    func incrementedBy(_ value: Int) -> CoWInt {
        let newStorage = heapStorage.copy()
        newStorage.value += value
        return Self(storage: newStorage)
    }
}

func myFunc6() {
    var a = CoWInt(3)
    print(a)
    a = a.incrementedBy(3)  // Forces a copy!
//     a += 3
    print(a)
}
myFunc6()

CoWInt(3, copyCount: 0)
CoWInt(6, copyCount: 1)


## Tuple Destructuring

Swift's value semantic types (`struct`s, `enum`s, and tuples) can be freely torn apart by the compiler during optimizations.

In [26]:
func myFunc7() {
    var t = (CoWInt(3), CoWInt(5))
    print(t)
    t.0 += 10
    print(t)
}
myFunc7()

(CoWInt(3, copyCount: 0), CoWInt(5, copyCount: 0))
(CoWInt(13, copyCount: 0), CoWInt(5, copyCount: 0))


In [27]:
func myFunc8() {
    var t = (a: CoWInt(3), b: CoWInt(5))
    print(t)
    var copy = t
    print(copy)
    t.1 += 5
    print(t)
    print(copy)
}
myFunc8()

(a: CoWInt(3, copyCount: 0), b: CoWInt(5, copyCount: 0))
(a: CoWInt(3, copyCount: 0), b: CoWInt(5, copyCount: 0))
(a: CoWInt(3, copyCount: 0), b: CoWInt(10, copyCount: 1))
(a: CoWInt(3, copyCount: 0), b: CoWInt(5, copyCount: 0))


## Swift's Law of Exclusivity

As detailed in the [ownership manifesto](https://github.com/apple/swift/blob/main/docs/OwnershipManifesto.md), Swift has a "law of exclusivity" that allows the compiler to make aggressive optimizations. Below are some examples showing how to violate Swift's law of exlusivity, and the resulting aftermath.

In [28]:
func incrementTwoInts(_ x: inout Int, _ y: inout Int) {
    x += 1
    y += 1
}

In [29]:
var a = 1
var b = 2
incrementTwoInts(&a, &b)
print("a: \(a), b: \(b)")

a: 2, b: 3


In [30]:
incrementTwoInts(&a, &a) // Compile error!

: 

In [31]:
extension Int {
    // Define a silly function that allows us to play around with closures.
    mutating func increment(by amount: (Self) -> Self) {
        let amt = amount(self)
        self += amt
    }
}

In [32]:
// Example usage of the API defined above.
print(a)
a.increment { $0 }  // equivalent to a + a
print(a)

2
4


In [33]:
func exclusivityViolationAtCompileTime() {
    var b = 3
    b.increment { bCopy in
      b = 0  // Exclusivity violation detected @ compile time!
      return bCopy
    }
}


: 

In [34]:
// Mutable globals are shared mutable state, so exclusivity detected at runtime.
a.increment { aCopy in
    a = 0  // Exlusivity violation!
    return aCopy
}

Simultaneous accesses to 0x7fc44a77c420, but modification requires exclusive access.
Previous access (a modification) started at  (0x7fc44a7729ed).
Current access (a modification) started at:
0    libswiftCore.so                    0x00007fc449bf04a0 swift_beginAccess + 479
Fatal access conflict detected.


: 